# important notice
This notebook **(05b)** is meant for running the network trained on the dataset in this directory on a dataset from a **different** directory. This can work if conditions are similar, pixelsize is the same. This is useful in case there is (1) no data tot train a network or (2) you just want some quick results.

You just need to have a tomogram ready (i.e. a .rec file)

In [ ]:
from train_cryo_care import CryoCARE
from generate_train_data import normalize, compute_mean_std, denormalize

import mrcfile
import numpy as np
import os
from os.path import basename, splitext

from matplotlib import pyplot as plt
from glob import glob

import subprocess

In [ ]:
# imports and settings for the GPU

os.environ["CUDA_VISIBLE_DEVICES"]="0" # <<< Set the GPU you want to use for the network here (0-7)

First we create a directory where you should move the .rec file of the external tomogram into. In principle you could also link directly to the file in the different directory, but this is cleaner. Before copying any files, run the code below.

You can now copy all the tomograms you want to have the network run on into `/external tomograms`. So your `/external_tomograms` can contain multiple .rec files. The script will process them automatically. The original file and the denoised file will be moved to `/external_tomograms/output` after processing. This means you can always copy in more .rec files later in into  `/external tomograms` without it accidentally reprocessing the same files.

In [ ]:
# Create the output directory
if not os.path.isdir('external_tomograms/'):
    os.mkdir('external_tomograms/')
if not os.path.isdir('external_tomograms/output/'):
    os.mkdir('external_tomograms/output/')

## Now we put the main process in a function

In [ ]:
def denoiseAndSave(tomogram_path):
    tomo_name = splitext(basename(tomogram_path))[0]
    tomo = mrcfile.open(glob(tomogram_path)[0]).data
    tomo_norm = normalize(tomo, mean, std)
    tomo_denoised = denormalize(model.predict(tomo_norm, axes='ZYX', n_tiles=(8,8,8), normalizer=None), mean, std)
    # Save the denoised tomogram.
    with mrcfile.open('external_tomograms/output/denoised_' + tomo_name + '.mrc', 'w+') as mrc:
        mrc.set_data(tomo_denoised)
    # move the .rec file to the output directory
    subprocess.run('mv ' + tomogram_path + ' external_tomograms/output/' + tomo_name + '.rec',shell = True)

## Now we denoise and move all the files 

In [ ]:
tomograms = glob('external_tomograms/*.rec') # get the paths of the tomograms
print(tomograms)

In [ ]:
# We load mean and standard deviation that where used to normalize the training data
mean_std = np.load('train_data/mean_std.npz')
mean, std = mean_std['mean'], mean_std['std']

# we load the model 
model = CryoCARE(None, 'denoiser_model', basedir='')

In [ ]:
for tomogram in tomograms:
    print('Denoising tomogram: ' + basename(tomogram) + '\n')
    denoiseAndSave(tomogram)

In [ ]:
# restart the kernel so the GPUs are freed - not a very elegant way
# this will result in a pop up dialog saying 'The kernel appears to have died. It will restart automatically.'
# you can ignore this and accept, the script should have completed successfully

quit